In [1]:
import pandas as pd
import numpy as np
import folium
import warnings
warnings.filterwarnings('ignore')


In [2]:

# 코드 목표 : 
# 월별 데이터, 포맷이 일정한것 확인 > 빈데이터 무시(dropna) > 지점별로 묶고(groupby) >
# 시간별 평균(소수점 반올림) > 빈데이터에 시간별 평균값 넣어주기 > 다시 평균 구하기 
# 지점별 위치 넣어주고 > 135개 지점 위치정보를 불러와서 지도화 > 가능하면 시계열성으로 보여주기

# 이걸 1월부터 12월까지 포맷or함수로 자동화


In [3]:
j=input('몇월의 데이터를 불러올까요?(01~12) >>>')

몇월의 데이터를 불러올까요?(01~12) >>> 04


In [4]:
df = pd.read_excel('project_01/{}월 서울시 교통량 조사자료.xlsx'.format(j), sheet_name='2021년 {}월'.format(j))

In [5]:
for i in range(1,24):
    df['{}시'.format(i)] = df.groupby(['지점번호'])['{}시'.format(i)].transform(lambda x: x.fillna(x.mean()))

In [6]:
df_na = df.fillna(0)

In [7]:
df_g = df_na.groupby('지점번호').mean().round().reset_index()

In [8]:
df_l = pd.read_excel('project_01/01월 서울시 교통량 조사자료.xlsx', sheet_name='수집지점 주소 및 좌표')

In [9]:
df_l_d = df_l.drop([df_l.index[135],df_l.index[136]])

In [10]:
df_l_pre = df_l_d[['지점번호','지점명칭','위도','경도']]

In [11]:
df_mg = pd.merge(df_g, df_l_pre)

In [12]:
s_loc = 37.566535, 126.9779692

s_map = folium.Map(s_loc, zoom_start=11) #location=중심좌표 -> [위도,경도]
i = input('시간을 입력하시오(1~23) >>')

for x in range(len(df_mg)):

    if df_mg['{}시'.format(i)][x] <= 1000:
        color = 'blue'
    elif df_mg['{}시'.format(i)][x] <= 2000:
        color = 'green'
    elif df_mg['{}시'.format(i)][x] <= 3000:    
        color = 'yellow'
    elif df_mg['{}시'.format(i)][x] <= 4000:    
        color = 'orange'
    else:
        color = 'red'  # 미확인

    folium.Marker([df_mg['위도'][x],df_mg['경도'][x]], popup=df_mg['지점명칭'][x], icon=folium.Icon(color = color)).add_to(s_map)

s_map

시간을 입력하시오(1~23) >> 17
